# Extract the header file

`cd ../input`    
`ln -s /Volumes/LACIE_SHARE/data/2020-06-10-10x-scATAC-seq/atac_pbmc_5k_v1_possorted_bam/atac_pbmc_5k_v1_possorted_bam.bam ./`    
`samtools view ./tac_pbmc_5k_v1_possorted_bam.bam -H > atac_v1_pbmc_5k_possorted_bam.header.sam`

# Create a bam file with the barcode embedded into the read name

`cat <( cat atac_v1_pbmc_5k_possorted_bam.header.sam ) \ <( samtools view atac_pbmc_5k_v1_possorted_bam.bam | awk '{for (i=12; i<=NF; ++i) { if ($i ~ "^CB:Z:"){ td[substr($i,1,2)] = substr($i,6,length($i)-5); } }; printf "%s:%s\n", td["CB"], $0 }' ) \
| samtools view -bS - > 10xpbmc5k.snap.bam` 

`samtools view 10xpbmc5k.snap.bam | cut -f 1 | head`

# sort the bam file by read name

`samtools sort -n -@ 15 -m 1G 10xpbmc5k.snap.bam -o 10xpbmc5k.snap.nsrt.bam`

# Obtain snap

`sh run_snaptools.sh`

# Obatain bin matrix

In [16]:
library(GenomicRanges)
library(SnapATAC)

`cp /Volumes/LACIE_SHARE/data/2020-06-10-10x-scATAC-seq/atac_pbmc_5k_v1_analysis/analysis/clustering/kmeans_8_clusters/clusters.csv ../input/metadata.tsv`

In [17]:
metadata <- read.table('../input/metadata.tsv',
                         header = TRUE,
                         stringsAsFactors=FALSE,quote="",row.names=1)

In [18]:
x.sp = createSnap(
file="../output/10xpbmc5k.snap",
sample="10xpbmc5k",
do.par = TRUE,
num.cores=10)

Epoch: reading the barcode session ...



#### Bin size selection (SnapATAC)

In [19]:
showBinSizes("../output/10xpbmc5k.snap");
x.sp = addBmatToSnap(x.sp, bin.size=5000, num.cores=10);

[1] 5000

Epoch: reading cell-bin count matrix session ...



#### Matrix binarization (SnapATAC)

In [20]:
x.sp = makeBinary(x.sp, mat="bmat");

####  Bin filtration (SnapATAC)

In [ ]:
# system("wget http://mitra.stanford.edu/kundaje/akundaje/release/blacklists/hg19-human/wgEncodeHg19ConsensusSignalArtifactRegions.bed.gz -O ../input/wgEncodeHg19ConsensusSignalArtifactRegions.bed.gz")

In [6]:
black_list = read.table('../input/wgEncodeHg19ConsensusSignalArtifactRegions.bed.gz')
black_list.gr = GRanges(black_list[,1], IRanges(black_list[,2], black_list[,3]));
idy1 = queryHits(findOverlaps(x.sp@feature, black_list.gr));
idy2 = grep("chrM|random", x.sp@feature);
idy = unique(c(idy1, idy2));
x.sp = x.sp[,-idy, mat="bmat"];

In [7]:
x.sp = filterBins(
x.sp,
low.threshold=-2,
high.threshold=2,
mat="bmat"
);

#### add full range to bmat

In [8]:
bmat<-x.sp@bmat
colnames(bmat)<-x.sp@feature@elementMetadata@listData$name

fullRange<-read.table("../input/10xpbmc5k_5k_full.txt")

bmat_full<-matrix(ncol = length(fullRange$V1),nrow = nrow(bmat))

rownames(bmat_full)<-rownames(bmat)
colnames(bmat_full)<-fullRange$V1

for(i in 1:length(fullRange$V1)){
    range<-as.character(fullRange$V1[i])
    if(range %in% colnames(bmat)){
        bmat_full[,range]<-bmat[,range]
    } else{
        bmat_full[,range]<-0
    }
}

In [16]:
saveRDS(bmat,file="../output/10xpbmc5k-snap.rds")
saveRDS(bmat_full,file="../output/10xpbmc5k-snap-full.rds")

   [[ suppressing 33 column names ‘chr1:10001-15000’, ‘chr1:15001-20000’, ‘chr1:55001-60000’ ... ]]



574 x 508175 sparse Matrix of class "dgCMatrix"
                                                                          
AAAACGTA . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AAAATGAA . . . . . . 1 . . . . . . . 1 . . . . . . . . . . . . . . . . 1 1
AAACACTT . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AAACCTTA . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AAACGCTC . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AAACTATG . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AAACTGGG . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AAAGGGGG . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AAAGTGGG . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AAAGTTCA . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AAATTCCT . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
AACATCGT . . . . . . . . . . . . . . . . . . . . . .